In [1]:
import onnx
from onnx import optimizer
import mxnet as mx
import mxnet.contrib.onnx as onnx_mxnet
from mxnet.gluon.data.vision import transforms

import numpy as np

In [2]:
# Pixel values of img must be already scalled to be in range [0,1]
def resize_crop_normalize_bachify(img):   
    transform_fn = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img = transform_fn(img)
    # Add one more axis to tensor 3x224x224 -> 1x3x224x224
    # This operation is called bachification
    img = img.expand_dims(axis=0)
    return img

def resize_crop_bachify(img):   
    transform_fn = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
    ])
    img = transform_fn(img)
    # Add one more axis to tensor 3x224x224 -> 1x3x224x224
    # This operation is called bachification
    img = img.expand_dims(axis=0)
    return img

def inference(model, argp, auxp):
    ex = model.bind(
        ctx=mx.cpu(),
        args = argp,
        aux_states= auxp
    )    
    ex.forward()
    return ex.outputs[0]

In [3]:
try:
    img = mx.image.imread('kitten.jpg')
except:
    mx.test_utils.download('https://s3.amazonaws.com/model-server/inputs/kitten.jpg')
    img = mx.image.imread('kitten.jpg')
# Shape of img is WxHx3    

In [4]:
def get_labels():
    with open('synset.txt', 'r') as f:
        labels = [l.rstrip() for l in f]    
    return labels    
try:
    labels= get_labels()
except:
    mx.test_utils.download('https://s3.amazonaws.com/onnx-model-zoo/synset.txt')
    labels= get_labels()


In [7]:
# Can take a long time
try:
    original_model, arg_params, aux_params  = onnx_mxnet.import_model('resnet152v2.onnx')
except:
    mx.test_utils.download('https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet152v2/resnet152v2.onnx')
    original_model, arg_params, aux_params  = onnx_mxnet.import_model('resnet152v2.onnx')

## Inference with original model

In [8]:
input_tensor = resize_crop_normalize_bachify(img)

arguments = arg_params.copy()
arguments.update(
    {
        'data' : input_tensor
    }
)

output_tensor = inference(original_model, arguments, aux_params)


In [9]:
scores = mx.nd.softmax(output_tensor).asnumpy()
scores = np.squeeze(scores)
a = np.argsort(scores)[::-1]
for i in a[0:5]:
    print('class=%s ; probability=%f' %(labels[i],scores[i]))

class=n02123045 tabby, tabby cat ; probability=0.724649
class=n02123159 tiger cat ; probability=0.267098
class=n02124075 Egyptian cat ; probability=0.007431
class=n02127052 lynx, catamount ; probability=0.000183
class=n02123394 Persian cat ; probability=0.000131


## Inference with preprocessing added 

In [10]:
# Create computational graph that standardizes the data
preprocess_net = mx.sym.Variable('input')
translation = mx.sym.Variable('preprocessing_translation', shape = (1,3,1,1), dtype=np.float32)
std = mx.sym.Variable('preprocessing_std', shape = (1,3,1,1), dtype=np.float32)

# Pixel values must also be translate and scalled by [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
preprocess_net = mx.sym.broadcast_sub(preprocess_net, translation)
preprocess_net = mx.sym.broadcast_div(preprocess_net, std)

preprocess_params = {
    'preprocessing_translation': mx.nd.array([0.485, 0.456, 0.406]).reshape((1,3,1,1)),
    'preprocessing_std': mx.nd.array([0.229, 0.224, 0.225]).reshape((1,3,1,1))              
}


In [11]:
model_with_preprocessing = original_model(data=preprocess_net)

In [12]:
input_tensor = resize_crop_bachify(img)

arguments = arg_params.copy()
arguments.update(
    {
        'input' : input_tensor 
    }
)
arguments.update(preprocess_params)

In [13]:
output_tensor = inference(model_with_preprocessing, arguments, aux_params)

In [14]:
scores = mx.nd.softmax(output_tensor).asnumpy()
scores = np.squeeze(scores)
a = np.argsort(scores)[::-1]
for i in a[0:5]:
    print('class=%s ; probability=%f' %(labels[i],scores[i]))

class=n02123045 tabby, tabby cat ; probability=0.724649
class=n02123159 tiger cat ; probability=0.267098
class=n02124075 Egyptian cat ; probability=0.007431
class=n02127052 lynx, catamount ; probability=0.000183
class=n02123394 Persian cat ; probability=0.000131


## Inference with postprocessing and preprocessing steps

In [15]:
full_model = mx.sym.softmax(data=model_with_preprocessing)
full_model = mx.sym.reshape(data=full_model, shape=(1000,))

In [16]:
output_tensor = inference(full_model, arguments, aux_params)
print(output_tensor.shape)

(1000,)


In [17]:
a = np.argsort(scores)[::-1]
for i in a[0:5]:
    print('class=%s ; probability=%f' %(labels[i],scores[i]))

class=n02123045 tabby, tabby cat ; probability=0.724649
class=n02123159 tiger cat ; probability=0.267098
class=n02124075 Egyptian cat ; probability=0.007431
class=n02127052 lynx, catamount ; probability=0.000183
class=n02123394 Persian cat ; probability=0.000131


 Looks like model produces correct results. We export it to ONNX format 

In [18]:
parameters = arg_params.copy()
parameters.update(aux_params)
parameters.update(preprocess_params)
onnx_mxnet.export_model(
    sym=full_model, 
    params = parameters, 
    input_shape=[(1, 3, 224, 224)],
    onnx_file_path = 'full_model.onnx'
)

'full_model.onnx'

## Adding denotation to the model

Denotationg inputs and outputs of full model. Input has type image. Output has categorical output type.   

In [19]:
# This can take a long time 
print('Loading model...')
model_proto = onnx.load('full_model.onnx')
print("Model loaded")

Loading model...
Model loaded


In [20]:
graph_proto = model_proto.graph

### Type Denotation 

In [21]:
graph_proto.input[0].type.denotation = 'IMAGE'

In [22]:
graph_proto.output[0].type.denotation = 'CATEGORICAL_PROBABILITIES'


In [23]:
# Dimension Denotation

In [25]:
graph_proto.input[0].type.tensor_type.shape.dim[0].denotation = "DATA_BATCH"
graph_proto.input[0].type.tensor_type.shape.dim[1].denotation = "DATA_CHANNEL"
graph_proto.input[0].type.tensor_type.shape.dim[2].denotation = "DATA_FEATURE"
graph_proto.input[0].type.tensor_type.shape.dim[3].denotation = "DATA_FEATURE"

graph_proto.output[0].type.tensor_type.shape.dim[0].denotation = "DATA_PROBABILITY"

In [26]:
print(graph_proto.input[0])

name: "input"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 1
        denotation: "DATA_BATCH"
      }
      dim {
        dim_value: 3
        denotation: "DATA_CHANNEL"
      }
      dim {
        dim_value: 224
        denotation: "DATA_FEATURE"
      }
      dim {
        dim_value: 224
        denotation: "DATA_FEATURE"
      }
    }
  }
  denotation: "IMAGE"
}



In [27]:
print(graph_proto.output[0])

name: "reshape0"
type {
  tensor_type {
    elem_type: FLOAT
    shape {
      dim {
        dim_value: 1000
        denotation: "DATA_PROBABILITY"
      }
    }
  }
  denotation: "CATEGORICAL_PROBABILITIES"
}



### Model Metadata

In [28]:
meta_data_proto = model_proto.metadata_props

Metadata for input.

In [30]:
pixFormat = meta_data_proto.add()
pixFormat.key = "Image.BitmapPixelFormat"
pixFormat.value = "Rgb8"
colorSpace = meta_data_proto.add()
colorSpace.key = "Image.ColorSpaceGamma"
colorSpace.value = "SRGB"
nominalRange = meta_data_proto.add()
nominalRange.key = "Image.NominalPixelRange"
nominalRange.value = "Normalized_0_1"


In [31]:
print(model_proto.metadata_props)

[key: "Image.BitmapPixelFormat"
value: "Rgb8"
, key: "Image.ColorSpaceGamma"
value: "SRGB"
, key: "Image.NominalPixelRange"
value: "Normalized_0_1"
]


Metadata for output.

In [25]:
for i, label in enumerate(labels):
    category_part = label[10:]
    category = meta_data_proto.add()
    category.key = 'Category_' + str(i)
    category.value = category_part

In [26]:
print(meta_data_proto) 

[key: "Image.BitmapPixelFormat"
value: "Rgb8"
, key: "Image.ColorSpaceGamma"
value: "SRGB"
, key: "Image.NominalPixelRange"
value: "Normalized_0_1"
, key: "Category_0"
value: "tench, Tinca tinca"
, key: "Category_1"
value: "goldfish, Carassius auratus"
, key: "Category_2"
value: "great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias"
, key: "Category_3"
value: "tiger shark, Galeocerdo cuvieri"
, key: "Category_4"
value: "hammerhead, hammerhead shark"
, key: "Category_5"
value: "electric ray, crampfish, numbfish, torpedo"
, key: "Category_6"
value: "stingray"
, key: "Category_7"
value: "cock"
, key: "Category_8"
value: "hen"
, key: "Category_9"
value: "ostrich, Struthio camelus"
, key: "Category_10"
value: "brambling, Fringilla montifringilla"
, key: "Category_11"
value: "goldfinch, Carduelis carduelis"
, key: "Category_12"
value: "house finch, linnet, Carpodacus mexicanus"
, key: "Category_13"
value: "junco, snowbird"
, key: "Category_14"
value: "indigo bu

Other information

In [27]:
model_proto.producer_name = 'MXNet'
model_proto.producer_version = mx.__version__
model_proto.domain = 'com.ericsson.nomadiclab'
model_proto.model_version = 2
model_proto.doc_string = """ResNet152 v2 model for image classification.
Take an image as input and classifies the major object in the image into a set of 1000 predefined categories.
Model was trained in ImageNet dataset which contains images from 1000 categories.
Has 152 layers, excluding pre- and postprocessing.
"""
print(model_proto.producer_name)
print(model_proto.producer_version)
print(model_proto.domain)
print(model_proto.model_version)
print(model_proto.doc_string)

MXNet
1.3.1
com.ericsson.nomadiclab
2
ResNet152 v2 model for image classification.
Take an image as input and classifies the major object in the image into a set of 1000 predefined categories.
Model was trained in ImageNet dataset which contains images from 1000 categories.
Has 152 layers, excluding pre- and postprocessing.



Now the model is ready to be consumed by intelligence layer. We save it to disk.

In [28]:
onnx.save_model(model_proto, 'resnet152v2_prepostprocess_denotated.onnx')


In [29]:
tmp = onnx.load('resnet152v2_prepostprocess_denotated.onnx')

In [30]:
all_passes = optimizer.get_available_passes()
print("Available optimization passes:")
for p in all_passes:
    print(p)
print()

Available optimization passes:
eliminate_identity
eliminate_nop_pad
eliminate_nop_transpose
eliminate_unused_initializer
extract_constant_to_initializer
fuse_add_bias_into_conv
fuse_bn_into_conv
fuse_consecutive_squeezes
fuse_consecutive_transposes
fuse_transpose_into_gemm
lift_lexical_references
nop
split_init
split_predict



In [31]:
passes = ['eliminate_nop_pad']
optimized_model = optimizer.optimize(tmp, passes)

In [32]:
onnx.save_model(optimized_model, 'tmp.onnx')

In [33]:
print(optimized_model.opset_import)


[domain: ""
version: 8
]


In [34]:
print(optimized_model.graph.node[3].domain)

In [35]:

onnx.checker.check_model(tmp)
print('The model is checked!')

The model is checked!
